In [197]:
import pandas as pd
import ta
import datetime
from datetime import date
import math
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output
import san
import coinmetrics
import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.linear_model import TweedieRegressor
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import requests
import io
from pandas.io.json import json_normalize

In [198]:
##Bybt API for Derivatives Data
url = "https://open-api.bybt.com/api/pro/v1/futures/funding_rates_chart?symbol=ETH"
bybtKey = ''
payload = {}
headers = {
  'bybtSecret': ''
}

response = requests.request("GET", url, headers=headers, data = payload)
response_text = response.text.encode('utf8')
response_text

b'{"code":"0","msg":"success","data":{"dateList":[1599667200000,1599696000000,1599724800000,1599753600000,1599782400000,1599811200000,1599840000000,1599868800000,1599897600000,1599926400000,1599955200000,1599984000000,1600012800000,1600041600000,1600070400000,1600099200000,1600128000000,1600156800000,1600185600000,1600214400000,1600243200000,1600272000000,1600300800000,1600329600000,1600358400000,1600387200000,1600416000000,1600444800000,1600473600000,1600502400000,1600531200000,1600560000000,1600588800000,1600617600000,1600646400000,1600675200000,1600704000000,1600732800000,1600761600000,1600790400000,1600819200000,1600848000000,1600876800000,1600905600000,1600934400000,1600963200000,1601020800000,1601049600000,1601078400000,1601107200000,1601136000000,1601164800000,1601193600000,1601222400000,1601251200000,1601280000000,1601308800000,1601337600000,1601366400000,1601395200000,1601424000000,1601452800000,1601481600000,1601510400000,1601539200000,1601568000000,1601596800000,160162560000

In [199]:
#reads in funding rate json response from Bybt, converts to dataframe then extracts dictionary items and converts that to series
fundingRate = pd.read_json(response_text,orient='columns',typ='frame')
fundingC = pd.DataFrame(fundingRate['data'])
fundingC = fundingC.T
fundingdateList = fundingC['dateList']
fundingdateList = fundingdateList.explode()
fundingdateList = fundingdateList.reset_index(drop=True)
fundingdataMap = fundingC['dataMap']
name, values = list(fundingdataMap.items())[0]
fundingdataMapF = pd.Series(name=name, data=values)
binance_funding = fundingdataMapF['Binance']
binance_funding = pd.Series(data=binance_funding)
binance_funding
fundingdateList

0      1599667200000
1      1599696000000
2      1599724800000
3      1599753600000
4      1599782400000
           ...      
500    1614124800000
501    1614153600000
502    1614182400000
503    1614211200000
504    1615235491930
Name: dateList, Length: 505, dtype: object

In [200]:
fund_chart = {'dateList': fundingdateList,'fundingRate':binance_funding}
fund_chart = pd.DataFrame(fund_chart)
l_fundingRate = fund_chart.iloc[-1,1]


In [144]:
##1hLS Chart
LSurl = 'https://open-api.bybt.com/api/pro/v1/futures/longShort_chart?interval=2&symbol=ETH'
LSrequest = requests.request("GET", LSurl, headers=headers, data = payload)
LSresponse = LSrequest.text.encode('utf8')
print(LSresponse)
LSdata = pd.read_json(LSresponse,orient='columns',typ='frame')

b'{"code":"0","msg":"success","data":{"symbol":"ETH","longRateList":[47.8500,44.5100,51.3300,46.8800,48.3900,52.3200,51.6500,43.2400,45.5700,47.6800,53.3700,45.7500,54.3100,52.2700,45.8400,53.7800,50.2000,50.3900,49.8600,51.3500,49.9400,50.9700,47.0900,42.5300,50.0400,47.7000,48.0200,54.1000,54.6300],"shortsRateList":[52.1500,55.4900,48.6700,53.1200,51.6100,47.6800,48.3500,56.7600,54.4300,52.3200,46.6300,54.2500,45.6900,47.7300,54.1600,46.2200,49.8000,49.6100,50.1400,48.6500,50.0600,49.0300,52.9100,57.4700,49.9600,52.3000,51.9800,45.9000,45.3700],"priceList":[1620.44000000,1606.28000000,1578.42000000,1595.17000000,1585.64000000,1568.21000000,1592.86000000,1613.54000000,1598.09000000,1555.75000000,1538.23000000,1561.39000000,1551.15000000,1573.25000000,1571.94000000,1548.14000000,1559.00000000,1563.00000000,1562.73000000,1567.92000000,1574.55000000,1577.67000000,1587.46000000,1562.67000000,1509.04000000,1527.13000000,1531.42000000,1517.50000000,1530.99000000],"longShortRateList":[0.9175

In [206]:
#1h
LS = pd.DataFrame(LSdata['data'])
LS = LS.T
LSdateList = LS['dateList']
LSdateList = LSdateList.explode()
LSdateList = LSdateList.reset_index(drop=True)
LSratio = LS['longShortRateList'].explode()
hLSratio = LSratio.reset_index(drop=True)
LSprice = LS['priceList']
LSprice = LSprice.explode()
LSprice = LSprice.reset_index(drop=True)
LSprice

0     1620.44
1     1606.28
2     1578.42
3     1595.17
4     1585.64
5     1568.21
6     1592.86
7     1613.54
8     1598.09
9     1555.75
10    1538.23
11    1561.39
12    1551.15
13    1573.25
14    1571.94
15    1548.14
16       1559
17       1563
18    1562.73
19    1567.92
20    1574.55
21    1577.67
22    1587.46
23    1562.67
24    1509.04
25    1527.13
26    1531.42
27     1517.5
28    1530.99
Name: priceList, dtype: object

In [146]:
LSchart = {'dateList': LSdateList,'LS Ratio':hLSratio}
LSchart = pd.DataFrame(LSchart)
LSratio = LSchart.iloc[-1,1]
LSratio

1.2041

In [192]:
#LS daily
DLSurl = 'https://open-api.bybt.com/api/pro/v1/futures/longShort_chart?interval=5&symbol=ETH'
DLSrequest = requests.request("GET", DLSurl, headers=headers, data = payload)
DLSresponse = DLSrequest.text.encode('utf8')
print(DLSresponse)
DLSdata = pd.read_json(DLSresponse,orient='columns',typ='frame')
DLS = pd.DataFrame(DLSdata['data'])
DLS = DLS.T
DLSdateList = DLS['dateList']
DLSdateList = DLSdateList.explode()
DLSdateList = DLSdateList.reset_index(drop=True)
DLSdateList = pd.to_datetime(DLSdateList,unit='ms',origin='unix')
DLSratio = DLS['longShortRateList'].explode()
DLSratio = DLSratio.reset_index(drop=True)
DLSratio
DLSchart = {'dateList': DLSdateList,'DLS Ratio':DLSratio}
DLSchart = pd.DataFrame(DLSchart)
cDLSratio = DLSchart.iloc[-1,1]
DLSchart

b'{"code":"0","msg":"success","data":{"symbol":"ETH","longRateList":[48.3100,49.7200,48.3600,46.9300,50.0000,48.6600,48.3400,49.2100,48.4700,48.8800,47.9400,48.3400,48.1100,48.7100,49.7700,48.5700,48.3000,47.6500,47.1600,48.7200,49.8000,49.7300,49.2900,48.1400,48.8500,51.6000,49.5400,49.4600,49.3200],"shortsRateList":[51.6900,50.2800,51.6400,53.0700,50.0000,51.3400,51.6600,50.7900,51.5300,51.1200,52.0600,51.6600,51.8900,51.2900,50.2300,51.4300,51.7000,52.3500,52.8400,51.2800,50.2000,50.2700,50.7100,51.8600,51.1500,48.4000,50.4600,50.5400,50.6800],"priceList":[1599.35000000,1587.63000000,1743.99000000,1674.34000000,1547.01000000,1731.69000000,1746.16000000,1713.17000000,1801.09000000,1790.52000000,1820.44000000,1800.79000000,1814.63000000,1776.91000000,1818.08000000,1914.10000000,1933.90000000,1979.45000000,1965.86000000,1733.27000000,1574.16000000,1661.98000000,1628.18000000,1535.28000000,1452.14000000,1317.33000000,1552.54000000,1554.05000000,1602.45000000],"longShortRateList":[0.9346

,dateList,DLS Ratio
0,2021-02-03 16:00:00,0.9346
1,2021-02-04 16:00:00,0.9889
2,2021-02-05 16:00:00,0.9365
3,2021-02-06 16:00:00,0.8843
4,2021-02-07 16:00:00,1
5,2021-02-08 16:00:00,0.9478
6,2021-02-09 16:00:00,0.9357
7,2021-02-10 16:00:00,0.9689
8,2021-02-11 16:00:00,0.9406
9,2021-02-12 16:00:00,0.9562


In [193]:
#Open Interest
OIurl = 'https://open-api.bybt.com/api/pro/v1/futures/openInterest/chart?interval=0&symbol=ETH'
OIresponse = requests.request("GET", OIurl, headers=headers, data = payload)
OIresponse_text = OIresponse.text.encode('utf8')
pd.read_json(OIresponse_text,orient='columns',typ='frame')

,code,msg,data,success
dataMap,0,success,"{'Binance': [49547303, 51082691, 52543626, 463...",True
dateList,0,success,"[1582819200000, 1582905600000, 1582992000000, ...",True
priceList,0,success,"[230.85, 227.35, 224.95, 220.35, 228.5, 224.95...",True


In [194]:
fundingURL = 'https://open-api.bybt.com/api/pro/v1/futures/funding_rates?symbol=ETH'
fund_response = requests.request("GET", fundingURL, headers=headers, data = payload)
fund_response_text = response.text.encode('utf8')
pd.read_json(fund_response_text,orient='columns',typ='frame')

,code,msg,data,success
dataMap,0,success,"{'Huobi': [-0.006162, -0.028366000000000002, -...",True
dateList,0,success,"[1599321600000, 1599350400000, 1599379200000, ...",True
priceList,0,success,"[357.14, 334.7, 340.12, 344.3, 352.8, 343.43, ...",True


In [150]:
from kucoin.client import Client
client = Client("5ec45879d3c2900008e7ffe7", "641fd5f2-edb0-4873-82c2-a4f2768ff241", "winning")

cmc = CoinMarketCapAPI('2e5382bc-3fc0-4433-8ea5-9518f6ccf297')


cg = CoinGeckoAPI()
#sets beginning and ending timestamp
now = datetime.datetime.now()
nowtime = now.timestamp()
nowML = nowtime/1000.0
today = date.today()
start1 = now - datetime.timedelta(days=2)
starttime = start1.timestamp()
startDay = now - datetime.timedelta(days=300)
starttimeML = startDay.timestamp()
startML = starttimeML/1000.0
endTime = math.floor(nowtime)
beginTime2d = math.floor(starttime)
beginTimeML = math.floor(starttimeML)
beginDate = date.fromtimestamp(beginTimeML)
print(beginDate)
print(endTime)

2020-05-09
1614904740


In [176]:
kline1d = client.get_kline_data('ETH-USDT','1day',beginTimeML)
data1 = pd.DataFrame(kline1d)
data1d = data1[[0,2]]
#data1d[0] = pd.to_datetime(data1h[0],unit='s',origin='unix')
data1d[2] = data1d[2].astype(float)
data1d = data1d.reindex(index=data1d.index[::-1])
closepriceML = data1d[2]
closepriceML
print(data1d)

              0        2
299  1589068800   187.63
298  1589155200   185.73
297  1589241600   189.80
296  1589328000   199.61
295  1589414400   203.18
..          ...      ...
4    1614556800  1570.64
3    1614643200  1487.13
2    1614729600  1567.39
1    1614816000  1538.89
0    1614902400  1464.39

[300 rows x 2 columns]


<ipython-input-176-4162ebf6dc6a>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [152]:
#FEATURE SETS
#market cap
#network momentum
#NVT Ratio
#Transaction Volume
#Price
kline1h = client.get_kline_data('ETH-USDT','1hour',beginTimeML,endTime)
data1h = pd.DataFrame(kline1h)
data1h = data1h[[0,2]]
data1h[0] = pd.to_datetime(data1h[0],unit='s',origin='unix')
data1h[2] = data1h[2].astype(float)
data1h = data1h.reindex(index=data1h.index[::-1])
closepriceML1 = data1h[2]
curr_price = data1h.iloc[0]
print(curr_price)
print(data1h)

0    2021-01-01 13:00:00
2                 740.08
Name: 1499, dtype: object
                       0        2
1499 2021-01-01 13:00:00   740.08
1498 2021-01-01 14:00:00   738.35
1497 2021-01-01 15:00:00   735.51
1496 2021-01-01 16:00:00   735.80
1495 2021-01-01 17:00:00   728.15
...                  ...      ...
4    2021-03-04 20:00:00  1530.99
3    2021-03-04 21:00:00  1518.16
2    2021-03-04 22:00:00  1529.83
1    2021-03-04 23:00:00  1538.89
0    2021-03-05 00:00:00  1484.05

[1500 rows x 2 columns]


In [153]:
#MarketCap
cm = coinmetrics.Community()
asset = "eth"
available_data_types = cm.get_available_data_types_for_asset(asset)
print(available_data_types)
#NVTAdj,NVTAdj90
metrics = "CapMrktCurUSD"
begin_timestamp = beginDate  # The `datetime` type is also accepted
end_timestamp = today
asset_data = cm.get_asset_metric_data(asset, metrics, begin_timestamp, end_timestamp,time_agg = 'hour')
pandasKNC = coinmetrics.cm_to_pandas(asset_data)
pandasKNC.reset_index(drop=True,inplace = True)
pandasKNC['CapMrktCurUSD'] = pandasKNC['CapMrktCurUSD'].astype(int)
print(pandasKNC)
pandasMarket = pandasKNC['CapMrktCurUSD']
curr_cap = pandasKNC['CapMrktCurUSD'].iloc[-1]
curr_cap

['AdrActCnt', 'BlkCnt', 'BlkSizeByte', 'BlkSizeMeanByte', 'CapMrktCurUSD', 'DiffMean', 'FeeMeanNtv', 'FeeMeanUSD', 'FeeMedNtv', 'FeeMedUSD', 'FeeTotNtv', 'FeeTotUSD', 'HashRate', 'IssContNtv', 'IssContPctAnn', 'IssContUSD', 'IssTotNtv', 'IssTotUSD', 'NVTAdj', 'NVTAdj90', 'PriceBTC', 'PriceUSD', 'ROI1yr', 'ROI30d', 'SplyCur', 'SplyExpFut10yrCMBI', 'SplyFF', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjNtv', 'TxTfrValAdjUSD', 'TxTfrValMeanNtv', 'TxTfrValMeanUSD', 'TxTfrValMedNtv', 'TxTfrValMedUSD', 'TxTfrValNtv', 'TxTfrValUSD', 'VtyDayRet180d', 'VtyDayRet30d', 'VtyDayRet60d']
     CapMrktCurUSD
0      23366876239
1      20793867006
2      20593814372
3      21051379335
4      22176401180
..             ...
294   166995249129
295   163724417567
296   179579378092
297   171172699837
298   182015174257

[299 rows x 1 columns]


182015174257

In [154]:
#BTC Dominance
globalCap = cg.get_global()
globalCap1 = pd.DataFrame(globalCap)
BTCdominance = globalCap1.iloc[0,7]
dominanceChange=globalCap1.iloc[0,8]


In [155]:
cm.get_asset_metric_data(asset, metrics, begin_timestamp, end_timestamp,time_agg = 'hour')

{'metrics': ['CapMrktCurUSD'],
 'series': [{'time': '2020-05-09T00:00:00.000Z',
   'values': ['23366876239.73545411689078635']},
  {'time': '2020-05-10T00:00:00.000Z',
   'values': ['20793867006.47762614339420035']},
  {'time': '2020-05-11T00:00:00.000Z',
   'values': ['20593814372.68382565671304846']},
  {'time': '2020-05-12T00:00:00.000Z',
   'values': ['21051379335.26808483793941525']},
  {'time': '2020-05-13T00:00:00.000Z',
   'values': ['22176401180.50613020016425461']},
  {'time': '2020-05-14T00:00:00.000Z',
   'values': ['22577916785.99270810804989248']},
  {'time': '2020-05-15T00:00:00.000Z',
   'values': ['21647546818.09218459322583868']},
  {'time': '2020-05-16T00:00:00.000Z',
   'values': ['22282398179.71225244941477278']},
  {'time': '2020-05-17T00:00:00.000Z',
   'values': ['22979026769.8292548842813172']},
  {'time': '2020-05-18T00:00:00.000Z',
   'values': ['23833700453.19965766813411559']},
  {'time': '2020-05-19T00:00:00.000Z',
   'values': ['23733631010.96466379099324

In [156]:
# Transaction Value

metrics = "TxTfrValAdjUSD"
begin_timestamp = beginDate  # The `datetime` type is also accepted
end_timestamp = today
TValue_data = cm.get_asset_metric_data(asset, metrics, begin_timestamp, end_timestamp)
pandasTValue = coinmetrics.cm_to_pandas(TValue_data)
pandasTValue.reset_index(drop=True,inplace = True)
pandasTValue['TxTfrValAdjUSD'] = pandasTValue['TxTfrValAdjUSD'].astype(int)
pandasTVML = pandasTValue['TxTfrValAdjUSD']
MA_TValue = pandasTValue['TxTfrValAdjUSD'].mean()
MA_TValue = int(MA_TValue)
print(pandasTVML)
curr_TValue = pandasTValue['TxTfrValAdjUSD'].iloc[-1]

0       309126037
1       612365725
2       452285941
3       458331915
4       503999362
          ...    
294    5592950934
295    9336045655
296    8510612789
297    7117712864
298    6033303732
Name: TxTfrValAdjUSD, Length: 299, dtype: int64


In [157]:
#NVTAdj,NVTAdj90
metrics = "NVTAdj"
begin_timestamp = beginDate  # The `datetime` type is also accepted
end_timestamp = today
NVT_data = cm.get_asset_metric_data(asset, metrics, begin_timestamp, end_timestamp)
pandasNVT = coinmetrics.cm_to_pandas(NVT_data)
pandasNVT.reset_index(drop=True,inplace = True)
pandasNVTML = pandasNVT['NVTAdj']
curr_NVT = pandasNVT['NVTAdj'].iloc[-1]
print(pandasNVTML)

0      75.590126
1      33.956615
2      45.532732
3      45.930424
4      44.000852
         ...    
294    29.858165
295    17.536806
296    21.100640
297    24.048835
298    30.168409
Name: NVTAdj, Length: 299, dtype: float64


In [158]:
#percentage change
stats = client.get_24hr_stats('KNC-BTC')
hrstats24 = pd.DataFrame(stats, index =[0])
percentage_change24 = hrstats24.iloc[0,4]
print(percentage_change24)

0.1284


In [159]:
ema = ta.trend.ema_indicator(closepriceML,n=25,fillna=True)
lastEMA = ema.iloc[-1]
rsi = ta.momentum.rsi(close = closepriceML, n=14,fillna=False)
rsi.reindex(index=rsi.index[::-1])
lastRSI = rsi.iloc[-1]
secondRSI = rsi.iloc[-2]
trend = (ema - closepriceML) / closepriceML
lastTrend = trend.iloc[-1]
rsi.head(20)

299          NaN
298          NaN
297          NaN
296          NaN
295          NaN
294          NaN
293          NaN
292          NaN
291          NaN
290          NaN
289          NaN
288          NaN
287          NaN
286    59.796695
285    52.032767
284    55.925524
283    52.467592
282    58.746371
281    66.549986
280    66.701725
Name: rsi, dtype: float64

In [160]:
globalBTC = cg.get_global()
globalBTC
#pd.DataFrame(globalBTC)

{'active_cryptocurrencies': 6357,
 'upcoming_icos': 0,
 'ongoing_icos': 50,
 'ended_icos': 3375,
 'markets': 575,
 'total_market_cap': {'btc': 31370366.27308319,
  'eth': 990115768.3916696,
  'ltc': 8519829016.258428,
  'bch': 3011031526.355821,
  'bnb': 6652069035.711995,
  'eos': 405108289286.6885,
  'xrp': 3196341955046.017,
  'xlm': 3704235889480.632,
  'link': 55245743485.429115,
  'dot': 43457771350.754,
  'yfi': 47486585.52864016,
  'usd': 1484178758999.0881,
  'aed': 5451378192552.345,
  'ars': 134020154594610.3,
  'aud': 1923866716352.567,
  'bdt': 125811236087524.27,
  'bhd': 559526487070.1014,
  'bmd': 1484178758999.0881,
  'brl': 8414998211951.774,
  'cad': 1881868910009.1702,
  'chf': 1379364570859.814,
  'clp': 1086566536278924.9,
  'cny': 9603081824351.793,
  'czk': 32565256658454.348,
  'dkk': 9224497506506.303,
  'eur': 1240400913654.7292,
  'gbp': 1068322259978.8569,
  'hkd': 11519230811157.572,
  'huf': 452943528894790.7,
  'idr': 2.1184832275056696e+16,
  'ils': 491

In [179]:
#Coin Gecko Data
deriv = cg.get_derivatives()
derivatives = pd.DataFrame(deriv)
derivatives.head(10)
ETHfundingrate = derivatives.iloc[5,9]
derivatives

,market,symbol,index_id,price,price_percentage_change_24h,contract_type,index,basis,spread,funding_rate,open_interest,volume_24h,last_traded_at,expired_at
0,Bybit,BTCUSD,BTC,46862.5,-7.830816,perpetual,46840.470000,-0.047010,0.01,0.080,1.833654e+09,8.558684e+09,1614906214,NaN
1,Binance (Futures),BTCUSDT,BTC,46662.27,-7.803099,perpetual,46810.802340,-0.034334,0.01,0.010,1.394803e+09,1.876753e+10,1614906230,NaN
2,FTX (Derivatives),BTC-PERP,BTC,46812.0,-7.850394,perpetual,46845.908848,0.072436,0.01,0.036,9.760682e+08,3.793323e+09,1614906222,NaN
3,BitMEX,XBTUSD,BXBT,46834.0,-8.249584,perpetual,46864.840000,0.065850,0.01,0.010,7.694393e+08,2.717439e+09,1614906208,NaN
4,Binance (Futures),BTCUSD_PERP,BTC,46813.2,-7.836602,perpetual,46801.683545,-0.024601,0.01,0.010,6.828433e+08,6.511470e+09,1614906229,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,FTX (Derivatives),USO-0326,USO,43.17,4.654545,futures,43.080000,-0.208478,0.35,0.000,NaN,2.291227e+04,1614890589,1.616789e+09
1451,FTX (Derivatives),WSB-0326,WSB,477.85,-2.935202,futures,468.258754,-2.007167,0.72,0.000,NaN,1.439600e+03,1614881969,1.616789e+09
1452,FTX (Derivatives),GDX-0326,GDX,31.35,1.571359,futures,31.275000,-0.239234,1.49,0.000,NaN,1.253700e+03,1614872864,1.616789e+09
1453,FTX (Derivatives),BB-0326,BB,9.84,0.000000,futures,9.840000,0.000000,5.46,0.000,NaN,6.002400e+02,1614849732,1.616789e+09


In [162]:
derivatives.head(10)

,market,symbol,index_id,price,price_percentage_change_24h,contract_type,index,basis,spread,funding_rate,open_interest,volume_24h,last_traded_at,expired_at
0,Bybit,BTCUSD,BTC,46996.5,-7.123377,perpetual,47006.310000,0.020874,0.01,0.080000,1.839726e+09,8.513314e+09,1614904502,NaN
1,Binance (Futures),BTCUSDT,BTC,46993.48,-6.754499,perpetual,47017.670175,-0.026217,0.01,0.010000,1.405751e+09,1.864345e+10,1614904273,NaN
2,FTX (Derivatives),BTC-PERP,BTC,47097.0,-6.821644,perpetual,47071.074793,-0.055046,0.01,-0.043200,1.000218e+09,3.735373e+09,1614904455,NaN
3,BitMEX,XBTUSD,BXBT,47043.5,-6.967063,perpetual,47086.380000,0.091150,0.01,0.010000,7.679515e+08,2.691415e+09,1614904399,NaN
4,Binance (Futures),BTCUSD_PERP,BTC,47005.1,-6.864713,perpetual,47000.714545,-0.009330,0.01,0.010000,6.867726e+08,6.439543e+09,1614904274,NaN
5,BTSE (Futures),BTCPFC,BTCPFC,46975.0,-7.383675,perpetual,46960.666296,-0.030513,0.12,0.000000,6.485896e+08,4.894825e+08,1614904636,NaN
6,Bitfinex (Futures),BTCF0:USTF0,BTC,47080.0,-6.847906,perpetual,47096.513333,0.035075,0.01,-0.004919,5.665492e+08,4.406135e+07,1614904408,NaN
7,Deribit,BTC-PERPETUAL,BTC,47068.0,-6.969206,perpetual,47082.240000,0.030254,0.01,0.008214,5.554794e+08,1.346615e+09,1614904395,NaN
8,Binance (Futures),ETHUSDT,ETH,1496.1,-5.268486,perpetual,1496.060958,-0.080082,0.01,0.023638,5.448668e+08,4.721787e+09,1614904273,NaN
9,Bybit,ETHUSD,ETH,1493.0,-5.917197,perpetual,1492.040000,-0.064300,0.01,0.029600,4.946652e+08,2.073890e+09,1614904502,NaN


In [169]:
#x
curr_cap
curr_TValue
curr_velocity = curr_TValue/curr_cap #network momentum related to BTC bull runs
print(curr_price)
curr_NVT
NVT_signal = curr_cap/MA_TValue
lastEMA
lastRSI ##short term mean reversion metric for swing trading
lastTrend
l_fundingRate
binance_funding
cDLSratio
metrics_table = pd.DataFrame({'Close Price':closepriceML,'Cap':curr_cap,'Velocity':curr_velocity,'NVTS': NVT_signal, 
                              'NVT': curr_NVT, 'Percentage Change': percentage_change24,
                             'Trend Percentage': lastTrend, 'Funding Rate': l_fundingRate, 'LS Ratio': cDLSratio, 'Expo MA' : lastEMA, 'RSI' : lastRSI},index = [0]) 
#Velocity - netowrk momentum

#NVT Ratio - long term valuation metric

#MARKET SENTIMENT
##fear and greed index

#Options Expiry Date - suppression of price
#PCR
#Funding Rate of Futures - cm.get_market_info


#Crypto Flow
##BTC Season, AltCoin Season - boolean
#BTC Dominance a sign of money entering crypto markets - CoinMarketCap

#Identify walls (liquidity) in order books for levels, candle stick patterns
#KuCoin
print(metrics_table)
#y
percentage_change24


0    2021-01-01 13:00:00
2                 740.08
Name: 1499, dtype: object
   Close Price           Cap  Velocity       NVTS        NVT  \
0      1484.05  182015174257  0.033147  61.820608  30.168409   

  Percentage Change  Trend Percentage  Funding Rate  LS Ratio      Expo MA  \
0            0.1284          0.073006      0.023058    0.9732  1592.394885   

         RSI  
0  44.101367  


'0.1284'

In [196]:
data1d['Change'] = (closepriceML - closepriceML.shift(1)) / closepriceML * 100
daypctChange = data1d['Change']
velocity = (pandasTVML/pandasMarket) * 10
NVTsignal = pandasMarket/MA_TValue
#print(closepriceML,pandasMarket,pandasNVTML,pandasTVML,ema,rsi,trend)
xmetrics = pd.DataFrame({'ClosePrice':closepriceML,'TrendPct':trend,
                         'RSI' : rsi,'EMA' : ema,'Market Cap': pandasMarket,'NVT Signal': NVTsignal, 
                         'NVT Ratio' : pandasNVTML,'Velocity': velocity,'Transaction Value': pandasTVML,
                         'Funding Rate': 'NAN', 'LSRatio': DLSratio,
                         'PctChange': daypctChange,'PriceDirection': 1})
#xmetrics = xmetrics.reindex(index=xmetrics.index[::-1])
xmetrics = xmetrics.assign(RSI=xmetrics.RSI.loc[::-1].reset_index(drop=True))
xmetrics = xmetrics.assign(EMA=xmetrics.EMA.loc[::-1].reset_index(drop=True))
xmetrics = xmetrics.assign(ClosePrice=xmetrics.ClosePrice.loc[::-1].reset_index(drop=True))
xmetrics = xmetrics.assign(PctChange=xmetrics.PctChange.loc[::-1].reset_index(drop=True))
xmetrics = xmetrics.assign(TrendPct=xmetrics.TrendPct.loc[::-1].reset_index(drop=True))
xmetrics = xmetrics.assign(DLSratio=xmetrics.LSRatio.loc[::-1].reset_index(drop=True))
xmetrics['PriceDirection'] = np.where(xmetrics['PctChange'] > 0,1,-1)
xmetrics
#y=f(x)

,ClosePrice,TrendPct,RSI,EMA,Market Cap,NVT Signal,NVT Ratio,Velocity,Transaction Value,Funding Rate,LSRatio,PctChange,PriceDirection,DLSratio
0,187.63,0.000000,NaN,187.630000,2.336688e+10,7.936451,75.590126,0.132292,3.091260e+08,NAN,0.9346,NaN,-1,NaN
1,185.73,0.009443,NaN,187.483846,2.079387e+10,7.062540,33.956615,0.294493,6.123657e+08,NAN,0.9889,-1.022990,-1,NaN
2,189.80,-0.011264,NaN,187.662012,2.059381e+10,6.994593,45.532732,0.219622,4.522859e+08,NAN,0.9365,2.144362,1,NaN
3,199.61,-0.055252,NaN,188.581088,2.105138e+10,7.150003,45.930424,0.217721,4.583319e+08,NAN,0.8843,4.914583,1,NaN
4,203.18,-0.066325,NaN,189.704081,2.217640e+10,7.532112,44.000852,0.227268,5.039994e+08,NAN,1,1.757063,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1570.64,0.031514,47.693737,1620.136421,1.637244e+11,55.608237,17.536806,0.570229,9.336046e+09,NAN,NaN,9.652753,1,1
296,1487.13,0.082558,43.751835,1609.905157,1.795794e+11,60.993301,21.100640,0.473919,8.510613e+09,NAN,NaN,-5.615514,-1,0.8843
297,1567.39,0.025038,48.184356,1606.634761,1.711727e+11,58.138012,24.048835,0.415821,7.117713e+09,NAN,NaN,5.120615,1,0.9365
298,1538.89,0.040636,46.774792,1601.423625,1.820152e+11,61.820608,30.168409,0.331473,6.033304e+09,NAN,NaN,-1.851984,-1,0.9889


In [208]:
xmetrics['NVT Signal'].corr(xmetrics['PriceDirection'])

0.07822430294357056

In [198]:
#ML!!!!!!!
reg = linear_model.RidgeCV()

In [199]:
ytraining_set = xmetrics['PriceDirection']
ytraining_set = ytraining_set.loc[14:81]
xtraining_set = xmetrics.loc[14:81]
xtraining_set = xtraining_set.drop(columns = ['PriceDirection','NVT Signal','Market Cap','PctChange'])
reg.fit(xtraining_set,ytraining_set)
print(xtraining_set)
pd.DataFrame(reg.coef_).T

    ClosePrice  TrendPct        RSI         EMA  NVT Ratio  Velocity  \
14      170.38 -0.007737  51.044713  169.061844  75.436288  0.132562   
15      170.71 -0.008912  51.302755  169.188625  43.921849  0.227677   
16      182.71 -0.068312  59.634769  170.228731  49.232757  0.203117   
17      185.57 -0.076312  61.332846  171.408828  56.643481  0.176543   
18      187.60 -0.079668  62.537497  172.654303  50.142449  0.199432   
..         ...       ...        ...         ...        ...       ...   
77      243.13 -0.048265  59.275258  231.395442  45.192694  0.221275   
78      243.13 -0.044552  59.275258  232.298101  39.464487  0.253392   
79      234.60 -0.009057  52.132279  232.475170  38.467805  0.259958   
80      232.51 -0.000138  50.525707  232.477849  25.739451  0.388509   
81      229.62  0.011489  48.308758  232.258014  19.086763  0.523923   

    Transaction Value  
14          265682968  
15          430641753  
16          385302387  
17          356599261  
18          409

,0,1,2,3,4,5,6
0,0.070089,-0.088109,-0.057797,-0.072317,-0.012496,2.627278,-0.000004


In [519]:
reg.score(xtraining_set,ytraining_set)

-27037.148759327254

In [158]:
crypt_predict = reg.predict(xtraining_set)
pd.DataFrame(crypt_predict)

,0
0,-698.840036
1,-69.008742
2,-242.044702
3,-352.109881
4,-149.067237
...,...
63,429.265618
64,908.893037
65,973.851174
66,2168.694619
